In [27]:
from dotenv import load_dotenv,find_dotenv
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate,HumanMessagePromptTemplate

from langchain_community.callbacks.manager import get_openai_callback
from langchain.agents import AgentExecutor, create_tool_calling_agent, load_tools, initialize_agent,AgentType

In [28]:
load_dotenv(dotenv_path=find_dotenv(filename="../.env"))

True

In [3]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001").bind(logprobs=True)
llmOpenAI=ChatOpenAI(model="gpt-4o-mini").bind(logprobs=True)

<h3>Get Log Probabilities</h3>

In [4]:
# Does not work for Gemini

In [4]:
msg=llmGemini.invoke(input=("human","how are you today?"))

TypeError: GenerativeServiceClient.generate_content() got an unexpected keyword argument 'logprobs'

In [5]:
# Works for OpenAI Models
msg=llmOpenAI.invoke(input=("human","how are you today?"))

In [7]:
msg.response_metadata

{'token_usage': {'completion_tokens': 28,
  'prompt_tokens': 17,
  'total_tokens': 45,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_name': 'gpt-4o-mini-2024-07-18',
 'system_fingerprint': 'fp_34a54ae93c',
 'id': 'chatcmpl-BeJDK6lZlVI0dq8M0y7xjND5QpTax',
 'finish_reason': 'stop',
 'logprobs': {'content': [{'token': "I'm",
    'bytes': [73, 39, 109],
    'logprob': -0.04867241531610489,
    'top_logprobs': []},
   {'token': ' just',
    'bytes': [32, 106, 117, 115, 116],
    'logprob': -0.014044330455362797,
    'top_logprobs': []},
   {'token': ' a',
    'bytes': [32, 97],
    'logprob': -3.79811754100956e-05,
    'top_logprobs': []},
   {'token': ' computer',
    'bytes': [32, 99, 111, 109, 112, 117, 116, 101, 114],
    'logprob': -0.47782132029533386,
    'top_logprobs': []},
   {'token': ' program',
    '

In [9]:
msg.response_metadata["logprobs"]['content']

[{'token': "I'm",
  'bytes': [73, 39, 109],
  'logprob': -0.04867241531610489,
  'top_logprobs': []},
 {'token': ' just',
  'bytes': [32, 106, 117, 115, 116],
  'logprob': -0.014044330455362797,
  'top_logprobs': []},
 {'token': ' a',
  'bytes': [32, 97],
  'logprob': -3.79811754100956e-05,
  'top_logprobs': []},
 {'token': ' computer',
  'bytes': [32, 99, 111, 109, 112, 117, 116, 101, 114],
  'logprob': -0.47782132029533386,
  'top_logprobs': []},
 {'token': ' program',
  'bytes': [32, 112, 114, 111, 103, 114, 97, 109],
  'logprob': -5.9795445849886164e-05,
  'top_logprobs': []},
 {'token': ',',
  'bytes': [44],
  'logprob': -4.127333340875339e-06,
  'top_logprobs': []},
 {'token': ' so',
  'bytes': [32, 115, 111],
  'logprob': -0.07889000326395035,
  'top_logprobs': []},
 {'token': ' I',
  'bytes': [32, 73],
  'logprob': -1.9361264946837764e-07,
  'top_logprobs': []},
 {'token': " don't",
  'bytes': [32, 100, 111, 110, 39, 116],
  'logprob': -0.005234121344983578,
  'top_logprobs': [

In [10]:
sorted(msg.response_metadata["logprobs"]['content'],key=lambda s:s['logprob'],reverse=True)

[{'token': ' to', 'bytes': [32, 116, 111], 'logprob': 0.0, 'top_logprobs': []},
 {'token': ' I', 'bytes': [32, 73], 'logprob': 0.0, 'top_logprobs': []},
 {'token': ' you',
  'bytes': [32, 121, 111, 117],
  'logprob': 0.0,
  'top_logprobs': []},
 {'token': ' today',
  'bytes': [32, 116, 111, 100, 97, 121],
  'logprob': 0.0,
  'top_logprobs': []},
 {'token': '?', 'bytes': [63], 'logprob': 0.0, 'top_logprobs': []},
 {'token': ' I',
  'bytes': [32, 73],
  'logprob': -1.9361264946837764e-07,
  'top_logprobs': []},
 {'token': ' but',
  'bytes': [32, 98, 117, 116],
  'logprob': -3.128163257315464e-07,
  'top_logprobs': []},
 {'token': ' assist',
  'bytes': [32, 97, 115, 115, 105, 115, 116],
  'logprob': -7.896309739408025e-07,
  'top_logprobs': []},
 {'token': ',',
  'bytes': [44],
  'logprob': -4.127333340875339e-06,
  'top_logprobs': []},
 {'token': ' ready',
  'bytes': [32, 114, 101, 97, 100, 121],
  'logprob': -9.13388703338569e-06,
  'top_logprobs': []},
 {'token': ' feelings',
  'bytes'

In [9]:
msg.content

"I'm just a computer program, so I don't have feelings or emotions. How can I assist you today?"

<h3>LogProbs Also provides Token Counts</h3> 

In [13]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llmOpenAI=ChatOpenAI(model="gpt-3.5-turbo")

In [14]:
msg=llmOpenAI.invoke(input=("human","how are you today?"))

In [15]:
msg.response_metadata['token_usage']

{'completion_tokens': 38,
 'prompt_tokens': 17,
 'total_tokens': 55,
 'completion_tokens_details': {'accepted_prediction_tokens': 0,
  'audio_tokens': 0,
  'reasoning_tokens': 0,
  'rejected_prediction_tokens': 0},
 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}

<h3>Tracking Token Level Information</h3>

In [16]:
with get_openai_callback() as cb:
    result=llmGemini.invoke(input="Tell me a joke")
    print(cb)

Tokens Used: 21
	Prompt Tokens: 4
		Prompt Tokens Cached: 0
	Completion Tokens: 17
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $0.0


In [17]:
with get_openai_callback() as cb:
    result1=llmGemini.invoke(input="Tell me a joke")
    result2=llmOpenAI.invoke(input="Tell me a joke")
    print(cb)

Tokens Used: 49
	Prompt Tokens: 15
		Prompt Tokens Cached: 0
	Completion Tokens: 34
		Reasoning Tokens: 0
Successful Requests: 2
Total Cost (USD): $3.1e-05


In [49]:
with get_openai_callback() as cb:
    result1=llmGemini.invoke(input="Tell me a joke")
    result2=llmOpenAI.invoke(input="Tell me a joke")
    print(cb.total_tokens)

48


In [19]:

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You're a helpful assistant"),
        ("human", "{input}"),
        ("placeholder","{agent_scratchpad}")
    ]
)
tools=load_tools(tool_names=['wikipedia'])

In [20]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\MSI\\anaconda3\\envs\\lcnenv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000))]

In [21]:
agent=create_tool_calling_agent(llm=llmGemini,tools=tools,prompt=prompt)

In [22]:
agentExecutor=AgentExecutor(agent=agent,verbose=True,stream_runnable=False,tools=tools)

In [25]:
with get_openai_callback() as cb:
    response=agentExecutor.invoke(input=
                                  {"input":"Sachin Tendulkar?"}
                                 )
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost: {cb.total_cost}")



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Sachin Tendulkar'}`


Page: Sachin Tendulkar
Summary: Sachin Ramesh Tendulkar ( ; Marathi: [sətɕin t̪eɳɖulkəɾ]; born 24 April 1973) is an Indian former international cricketer who captained the Indian national team. He is widely regarded as one of the greatest cricketers of all time, and is the holder of several world records, including being the all-time highest run-scorer in both ODI and Test cricket, receiving the most player of the match awards in international cricket, and being the only batsman to score 100 international centuries. Tendulkar was a Member of Parliament, Rajya Sabha by presidential nomination from 2012 to 2018. 
Tendulkar took up cricket at the age of eleven, made his Test match debut on 15 November 1989 against Pakistan in Karachi at the age of sixteen, and went on to represent Mumbai domestically and India internationally for over 24 years. In 2002, halfway through his career, Wisden 

In [31]:
tools=load_tools(tool_names=["serpapi","llm-math"],llm=llmGemini)  # with wikipedia, it does not give the details
agent=create_tool_calling_agent(llm=llmGemini,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,verbose=True,stream_runnable=False,tools=tools)

In [32]:
with get_openai_callback() as cb:
    response=agentExecutor.invoke(input=
                                  {"input":"What's a hummingbird's scientific name and what's the fastest bird species?"}
                                 )
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost: {cb.total_cost}")



> Entering new AgentExecutor chain...

Invoking: `Search` with `What is the fastest bird species?`
responded: I am able to answer the first question.  I will need to search for the answer to the second question.

To answer your first question: I lack the specific knowledge to provide the scientific name of a hummingbird.  I would need access to a database of animal information to answer this question.

To answer your second question: I will need to use a search engine. I will use the available `default_api`'s `Search` function.

the Peregrine FalconI am not able to find the scientific name for a hummingbird, but the Peregrine Falcon is the fastest bird species.

> Finished chain.
Total Tokens: 299
Prompt Tokens: 169
Completion Tokens: 130
Total Cost: 0.0


<h3>Using an LLM for the same activity</h3>

In [33]:
tools=load_tools(tool_names=["serpapi","llm-math"],llm=llmGemini)

In [37]:
tools

[Tool(name='Search', description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='7f1f1a7b3b4c6064292064e174bd3d699c952fb4d998a31aafdd92b8e16edd64', aiosession=None)>, coroutine=<bound method SerpAPIWrapper.arun of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='7f1f1a7b3b4c6064292064e174bd3d699c952fb4d998a31aafdd92b8e16edd64', aiosession=None)>),
 Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['ques

In [34]:
agent=initialize_agent(tools=tools, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True,llm=llmGemini)

C:\Users\MSI\AppData\Local\Temp\ipykernel_15808\1306811143.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent=initialize_agent(tools=tools, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True,llm=llmGemini)


In [35]:
agentExecutor=AgentExecutor(agent=agent,verbose=True,stream_runnable=False,tools=tools)

In [36]:
with get_openai_callback() as cb:
    response=agent.invoke(
        input="Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?"
    )
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost: {cb.total_cost}")



> Entering new AgentExecutor chain...
Thought: I need to find out who Olivia Wilde's current boyfriend is and then calculate his age raised to the 0.23 power.  I'll use Search to find his name and age, and then Calculator to perform the calculation.

Action: Search
Action Input: "Olivia Wilde boyfriend"
Observation: As of early 2025, Wilde has been dating actor Dane DiLiegro.
Thought:Thought: I need to find Dane DiLiegro's age.
Action: Search
Action Input: "Dane DiLiegro age"
Observation: 36 years
Thought:Thought: I now have Dane DiLiegro's age. I need to calculate 36 to the power of 0.23.
Action: Calculator
Action Input: 36**0.23
Observation: Answer: 2.2800773226742175
Thought:I now know the final answer
Final Answer: Olivia Wilde's boyfriend, Dane DiLiegro, is 36 years old.  36 raised to the power of 0.23 is approximately 2.28.

> Finished chain.
Total Tokens: 1998
Prompt Tokens: 1778
Completion Tokens: 220
Total Cost: 0.0
